In [12]:
import numpy as np
import nibabel
from loadnii import segment_labels
import pymesh # https://github.com/pmneila/PyMCubes



### Segment Body parts

In [1]:
volume_file_path = "ctscan1/volume-3.nii.gz"
label_file_path = "ctscan1/labels-3.nii.gz"

labels = {"liver":1, "bladder":2, "lungs":3, "kidneys":4,"bones":5, "brain":6}

target_label = labels["bones"]
minlabel = target_label - .1
maxlabel = target_label + .1

new_image = segment_labels(volume_file_path, label_file_path, minlabel, maxlabel)
segmented_label_data = new_image.get_fdata()
print(f"Loaded volume of shape: {segmented_label_data.shape} Seperation value was: {minlabel} - {maxlabel}")


NameError: name 'segment_labels' is not defined

In [19]:
filename = "lungs"
nibabel.save(new_image, filename) # saving the file

### Marhcing Cubes 
My own implentation attempt

In [ ]:
cube_drawing = '''         y
         |        v7_______e6_____________v6
         |         /|                    /|
         |        / |                   / |
         |     e7/  |                e5/  |
         |      /___|______e4_________/   |
         |   v4|    |                 |v5 |e10
         |     |    |                 |   |
         |     |    |e11              |e9 |
         |   e8|    |                 |   |            z
         |     |    |_________________|___|           /
         |     |   / v3      e2       |   /v2        /
         |     |  /                   |  /          /
         |     | /e3                  | /e1        / 
         |     |/_____________________|/          /
         |     v0         e0          v1         / 
         |                                      /
         x-------------------------------------/'''


In [17]:



# The following data originates from Eric Lengyel's Transvoxel Algorithm.
# http://transvoxel.org/

In [ ]:

class Cube:
    cube_drawing = cube_drawing

    def __init__(self, index: int, 
                 v0: float, v1: float, v2: float, v3: float, 
                 v4: float, v5: float, v6: float, v7: float):
        self.v0 = v0
        self.v1 = v1
        self.v2 = v2
        self.v3 = v3
        self.v4 = v4
        self.v5 = v5
        self.v6 = v6
        self.v7 = v7
        self.cube_index = index
    
    def __iter__(self) -> tuple:
        return (self.v0, self.v1, self.v2, self.v3, self.v4, self.v5, self.v6, self.v7)
    
    def __contains__(self, item: float) -> bool:
        return item in self.__iter__()
    
    def __index__(self, index: int) -> float:
        return self.__iter__()[index]
    
    def __len__(self):
        return 8
    
    def __repr__(self):
        return f"Cube(index={self.cube_index}, v0={self.v0}, {self.v1}, {self.v2}, {self.v3}, {self.v4}, {self.v5}, {self.v6}, v7={self.v7})"
    
    @staticmethod
    def print_cube():
        """ Prints a character drawing of a cube and where the corners are """
        print(Cube.cube_drawing)


def march(volume) -> Cube:
    shape = volume.shape
    
    assert len(shape) == 3, f"The volume has to be 3 dimensions your volume has {len(shape)} dimensions"
    
    xlimit = shape[0]-2
    ylimit = shape[1]-2
    zlimit = shape[2]-2
    
    cube_index = 0
    
    for xindex, x in enumerate(volume):
        for yindex, y in enumerate(x):
            for zindex, z in enumerate(y):
                
                if zindex > zlimit or yindex > ylimit or xindex > xlimit:
                    break # Skip cube if its on the outside of the volume
                
                v0 = label_data[xindex][yindex][zindex] 
                v1 = label_data[xindex+1][yindex][zindex]
                v2 = label_data[xindex+1][yindex][zindex+1]
                v3 = label_data[xindex][yindex][zindex+1]
                v4 = label_data[xindex][yindex+1][zindex]
                v5 = label_data[xindex+1][yindex+1][zindex]
                v6 = label_data[xindex+1][yindex+1][zindex+1]
                v7 = label_data[xindex][yindex+1][zindex+1]
                
                yield Cube(cube_index, v0, v1, v2, v3, v4, v5, v6, v7)
                
                cube_index += 1



In [ ]:
print("Starting")
for cube in march(label4):
    pass
    # Check corners 
    # lookup in triangle table (I could not find this table)
print("Done")
                

### Marching cubes
skimage and mcubes implementation.

In [25]:
isovalue = 10
filename = "isovalue10"

In [ ]:
from skimage import measure
vertices, triangles, normals, values = measure.marching_cubes(label_data, isovalue)
meshskl = pymesh.form_mesh(vertices, triangles)

In [ ]:
import mcubes
vertices, triangles = mcubes.marching_cubes(smooth_label_data, iso)

### Smoothing

In [ ]:
print("Starting smoothing... this could take multile minutes")
smooth_label_data = mcubes.smooth(label_data)

In [ ]:
vertices, triangles = mcubes.marching_cubes(smooth_label_data, iso)
meshsmoothmcubes = pymesh.form_mesh(vertices, triangles)

In [ ]:
vertices, triangles, normals, values = measure.marching_cubes(label_data, isovalue)
meshsklsmooth = pymesh.form_mesh(vertices, triangles)

In [ ]:
# Saving with pymesh 
pymesh.save_mesh(f"{filename}mcubessmooth.obj", meshsmoothcubes)
pymesh.save_mesh(f"{filename}sklsmooth.obj", meshsklsmooth)

In [ ]:
# # example ball lamda function
# X, Y, Z = np.mgrid[:30, :30, :30]
# u = (X-15)**2 + (Y-15)**2 + (Z-15)**2 - 8**2
# vertices, triangles = mcubes.marching_cubes(u, 0)
# mcubes.export_mesh(vertices, triangles, "sphere.dae", "MySphere")